In [ ]:
from tqdm.notebook import tqdm, trange
from matplotlib.pyplot import imshow, subplots
import cv2
import numpy as np
import glob
import os
import pandas as pd
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_mse as mse
from skimage.measure import compare_psnr as psnr
from PIL import Image
from core.utils import ZipReader

In [ ]:
def quantify_results(Method, direc, SaveName, OrigFrameFolder, videoNames, options, zipped):
        #for cross change all ellipse to cross
        columnNames=["videoName", "Method", "PSNRavg","PSNRstd", "SSIMavg","SSIMstd", "MSEavg","MSEstd"]
        df = pd.DataFrame(columns=columnNames)            
        
        for videoName in tqdm(videoNames):
                TransFrameFiles=sorted(glob.glob(direc+videoName+options+"/*"))
                OrigFrameFile=OrigFrameFolder+videoName
                PSNR=[]
                MSE=[]
                SSIM=[]
                cnt=0
                adder=0
                for TransFrameFile in TransFrameFiles:
                        cnt=cnt+1
                        numStr=int(''.join(filter(str.isdigit, os.path.basename(TransFrameFile))))
                        if cnt==1 and numStr==1: adder=1
                        else: adder=0
                        TransName=(str(numStr-adder)).zfill(5)+".jpg"
                        TransImageFull= np.array(Image.open(TransFrameFile))
                        if zipped: OrigImage=np.array(ZipReader.imread(OrigFrameFile+".zip",TransName))
                        else: OrigImage=np.array(Image.open(OrigFrameFile+"/"+TransName))
                        PSNR.append(psnr(OrigImage,TransImageFull))
                        MSE.append(mse(OrigImage,TransImageFull))
                        SSIM.append(ssim(OrigImage,TransImageFull,multichannel=True))


                PSNRavg=np.mean(PSNR)
                PSNRstd=np.std(PSNR)

                MSEavg=np.mean(MSE)
                MSEstd=np.std(MSE)

                SSIMavg=np.mean(SSIM)
                SSIMstd=np.std(SSIM)

                df_temp=pd.DataFrame([[videoName, Method,
                                PSNRavg, PSNRstd, SSIMavg,SSIMstd, MSEavg,MSEstd]],columns=columnNames)
                # df=df.append(df_temp) # old pandas version
                df = pd.concat([df, df_temp]) # pandas higher than 2.0

        df.to_csv(SaveName+"quant_results.csv") 
        print("Saved results to",SaveName)

# Examples

In [ ]:

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Directory containing zipped test videos
zipped = True
OrigFrameFolder = "datasets/EndoSTTN_dataset/JPEGImages/"

# videos to be tested
videoNames=[f'hyperK_{str(i).zfill(3)}' for i in range(343, 373)]

# Directory containing inpainted results
inpainted_dir = "results/"
options = ["/nomask/B/frameresult/",
           "/nomask/B/frameresult/",
           "/nomask/frameresult/",
           "/nomask/frameresult/"]

# List of directories containing inpainted results using different trained models
inpainted_subdirs = ["CycleSTTN_model/gen_00002/full_video/",
                     "CycleSTTN_model/gen_00002/single_frame/",
                     "STTN_addition_model/gen_00003/full_video/",
                     "STTN_addition_model/gen_00003/single_frame/"]

# Directory to save your results
SaveDir = "Evaluation_Results/"

# For each trained model (method)
for index, inpainted_subdir in  enumerate(inpainted_subdirs):
    print("Processing", inpainted_subdir)
    os.makedirs(os.path.join(SaveDir, inpainted_subdir), exist_ok=True)
    direc=inpainted_dir+inpainted_subdir+"/"
    SaveName=SaveDir+inpainted_subdir
    quantify_results(inpainted_subdir, direc, SaveName, OrigFrameFolder, videoNames, options[index], zipped)
    

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Directory containing test videos
zipped = False
OrigFrameFolder = "datasets"

# videos to be tested
videoNames=['ExampleVideo1_Frames']

# Directory containing inpainted results
inpainted_dir = "results/"
options = ["/nomask/B/frameresult/",
           "/nomask/B/frameresult/",
           "/nomask/frameresult/",
           "/nomask/frameresult/"]

# List of directories containing inpainted results using different trained models
inpainted_subdirs = ["CycleSTTN_model/gen_00002/full_video/",
                     "CycleSTTN_model/gen_00002/single_frame/",
                     "STTN_addition_model/gen_00003/full_video/",
                     "STTN_addition_model/gen_00003/single_frame/"]

# Directory to save your results
SaveDir = "Evaluation_Results_NotInMainDataset/"

# For each trained model (method)
for index, inpainted_subdir in  enumerate(inpainted_subdirs):
    print("Processing", inpainted_subdir)
    os.makedirs(os.path.join(SaveDir, inpainted_subdir), exist_ok=True)
    direc=inpainted_dir+inpainted_subdir+"/"
    SaveName=SaveDir+inpainted_subdir
    quantify_results(inpainted_subdir, direc, SaveName, OrigFrameFolder, videoNames, options[index], zipped)
    